In [4]:
#pip install pyaudio

In [5]:
#pip install Pillow

In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import pyttsx3
import pandas as pd
from PIL import Image, ImageTk  # Import Pillow for handling jpg images

# Initialize pyttsx3 engine
engine = pyttsx3.init()

# List all available voices and set the desired voice
voices = engine.getProperty('voices')
for voice in voices:
    if 'en_IN' in voice.id:  # Check if the voice id includes 'en_IN' (Indian English)
        engine.setProperty('voice', voice.id)
        break

def speak(text):
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError:
            print("Sorry, my speech service is down.")
            return None

def fetch_train_data(source, destination, date):
    try:
        # Load the dataset from CSV
        df = pd.read_csv('scraped.csv')

        # Ensure the headers are present
        if not {'SRC_station', 'DESC_station', 'start_date'}.issubset(df.columns):
            raise ValueError("The CSV file is missing one of the required columns: 'SRC_station', 'DESC_station', 'start_date'")

        # Filter based on user input
        trains = df[(df['SRC_station'].str.lower() == source.lower()) & 
                    (df['DESC_station'].str.lower() == destination.lower()) & 
                    (df['start_date'] == date)]
        
        return trains if not trains.empty else None
    
    except KeyError as e:
        print(f"KeyError: {e} - Please check if the column names in the CSV match exactly with the ones in the code.")
        return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None



def generate_ticket(train_details, passenger_name, num_tickets):
    ticket_text = (f"Ticket Confirmation\n\n"
                   f"Passenger Name: {passenger_name}\n"
                   f"Train: {train_details['name'].values[0]}\n"
                   f"Date: {train_details['start_date'].values[0]}\n"
                   f"Source: {train_details['SRC_station'].values[0]}\n"
                   f"Destination: {train_details['DESC_station'].values[0]}\n"
                   f"Tickets: {num_tickets}\n\nSafe Journey!")
    return ticket_text


def print_ticket(ticket):
    # Simple print function (you can enhance it with a real print option)
    print(ticket)
    messagebox.showinfo("Ticket Printed", "Your ticket has been printed successfully!")

def Chatbot():
    def botReply():
        question = questionfield.get()
        process_query(question)
        questionfield.delete(0, END)
    
    def voiceQuery():
        question = listen()
        if question:
            questionfield.delete(0, END)
            questionfield.insert(END, question)
            botReply()
    
    def enterFunction(event):
        askbutton.invoke()

    def process_query(query):
        nonlocal state, passenger_name, source, destination, travel_date, num_tickets
        
        display_message('You', query)
        
        if state == 'initial':
            if 'book' in query:
                response = "Sure! Can I have your name, please?"
                state = 'name'
            else:
                response = "Hello! How can I assist you with your railway booking today?"
        
        elif state == 'name':
            passenger_name = query
            response = f"Thank you, {passenger_name}. Where would you like to travel from?"
            state = 'source'
        
        elif state == 'source':
            source = query
            response = "Got it. What's your destination?"
            state = 'destination'
        
        elif state == 'destination':
            destination = query
            response = "Perfect. Which date would you like to travel?"
            state = 'date'
        
        elif state == 'date':
            travel_date = query
            trains = fetch_train_data(source, destination, travel_date)
            if trains is not None:
                response = f"Great! I found {len(trains)} trains. How many tickets would you like to book?"
                state = 'tickets'
            else:
                response = "Sorry, no trains are available for this route and date. Please try another."
                state = 'source'
        
        elif state == 'tickets':
            num_tickets = query
            train_details = fetch_train_data(source, destination, travel_date)
            ticket = generate_ticket(train_details, passenger_name, num_tickets)
            response = ticket + "\n\nWould you like to print the ticket? (yes/no)"
            state = 'confirm'
        
        elif state == 'confirm':
            if 'yes' in query.lower():
                print_ticket(generate_ticket(fetch_train_data(source, destination, travel_date), passenger_name, num_tickets))
                response = "Your tickets have been booked and printed successfully! Have a safe journey!"
                state = 'initial'
            else:
                response = "No worries! You can book anytime later."
                state = 'initial'
        
        else:
            response = "I'm not sure how to respond to that."

        display_message('Bot', response)
        speak(response)
    
    def display_message(sender, message):
        msg_frame = Frame(text_frame, bg='#E1E8ED', bd=5, relief=GROOVE)
        msg_frame.pack(anchor='w' if sender == 'You' else 'e', padx=10, pady=5)
        
        Label(msg_frame, text=f"{sender}: {message}", font=('Arial', 12), bg='#ADD8E6' if sender == 'You' else '#FFD700', wraplength=350, justify=LEFT, bd=2, relief=SOLID).pack(fill=BOTH)

        text_frame.update_idletasks()
        text_canvas.config(scrollregion=text_canvas.bbox("all"))

    state = 'initial'
    passenger_name = ""
    source = ""
    destination = ""
    travel_date = ""
    num_tickets = ""

    root = tk.Tk()
    root.geometry('800x800')
    root.title("Railway Ticket Booking Chatbot")

    # Load and set background image
    bg_img = Image.open("railway.jpg")
    bg_img = bg_img.resize((800, 800), Image.ANTIALIAS)  # Resize to fit window
    bg_img = ImageTk.PhotoImage(bg_img)

    bg_label = Label(root, image=bg_img)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)

    # Logo (smaller size)
    logo_img = Image.open("logo.jpg")
    logo_img = logo_img.resize((100, 100), Image.ANTIALIAS)  # Resize to smaller size
    logo_img = ImageTk.PhotoImage(logo_img)
    logo_label = Label(root, image=logo_img, bg='#ADD8E6')
    logo_label.place(x=350, y=20)

    # Header
    header = Label(root, text="Railway Ticket Booking Bot", font=('Helvetica', 18, 'bold'), bg='#ADD8E6')
    header.place(x=200, y=150)

    # Center frame for chat area
    centerframe = Frame(root, bg='black')
    centerframe.place(relwidth=0.9, relheight=0.6, x=40, y=200)

    text_canvas = Canvas(centerframe, bg='black')
    text_canvas.pack(side=LEFT, fill=BOTH, expand=True)

    scrollbar = Scrollbar(centerframe, command=text_canvas.yview)
    scrollbar.pack(side='right', fill='y')

    text_canvas.config(yscrollcommand=scrollbar.set)
    
    text_frame = Frame(text_canvas, bg='black')
    text_canvas.create_window((0, 0), window=text_frame, anchor='nw')

    questionfield = Entry(root, font=('Verdana', 12), bd=5, relief=GROOVE)
    questionfield.place(relwidth=0.7, x=50, y=700)

    questionfield.bind("<Return>", enterFunction)

    askbutton = Button(root, text="Submit", font=('Helvetica', 12, 'bold'), bg='#4682B4', fg='white', relief=GROOVE, command=botReply)
    askbutton.place(x=650, y=695, width=100, height=30)

    #saybutton = Button(root, text="Say", font=('Helvetica', 12, 'bold'), bg='#4682B4', fg='white', relief=GROOVE, command=voiceQuery)
    #saybutton.place(x=650, y=730, width=100, height=30)
    
    root.mainloop()

Chatbot()


C:\Users\Komal Singh\AppData\Local\Temp\ipykernel_22236\954037168.py:177: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  bg_img = bg_img.resize((800, 800), Image.ANTIALIAS)  # Resize to fit window
C:\Users\Komal Singh\AppData\Local\Temp\ipykernel_22236\954037168.py:185: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  logo_img = logo_img.resize((100, 100), Image.ANTIALIAS)  # Resize to smaller size
